In [ ]:
import pymc as pm
import numpy as np
import pandas as pd
import arviz as az
import matplotlib.pyplot as plt
import math
import aesara.tensor as at

In [ ]:
train_data = pd.read_csv("data_acquisition/data_game_values_train.csv", sep=";")

In [ ]:
with pm.Model() as train_model:
    home_normal = pm.Normal("home_normal", shape=16)
    away_normal = pm.Normal("away_normal", shape=16)
    
    home_team = pm.Deterministic("home_team",   home_normal[0] * train_data["home_xG"] + 
                                                home_normal[1] * train_data["home_xT"] +
                                                home_normal[2] * train_data["home_xD"] +
                                                home_normal[3] * train_data["home_xK"] +
                                                home_normal[4] * train_data["home_sub_xG"] + 
                                                home_normal[5] * train_data["home_sub_xT"] +
                                                home_normal[6] * train_data["home_sub_xD"] +
                                                home_normal[7] * train_data["home_sub_xK"] +
                                                home_normal[8] * train_data["away_xG"] + 
                                                home_normal[9] * train_data["away_xT"] +
                                                home_normal[10] * train_data["away_xD"] +
                                                home_normal[11] * train_data["away_xK"] +
                                                home_normal[12] * train_data["away_sub_xG"] + 
                                                home_normal[13] * train_data["away_sub_xT"] +
                                                home_normal[14] * train_data["away_sub_xD"] +
                                                home_normal[15] * train_data["away_sub_xK"])

    away_team = pm.Deterministic("away_team",   away_normal[0] * train_data["away_xG"] + 
                                                away_normal[1] * train_data["away_xT"] +
                                                away_normal[2] * train_data["away_xD"] +
                                                away_normal[3] * train_data["away_xK"] +
                                                away_normal[4] * train_data["away_sub_xG"] + 
                                                away_normal[5] * train_data["away_sub_xT"] +
                                                away_normal[6] * train_data["away_sub_xD"] +
                                                away_normal[7] * train_data["away_sub_xK"] +
                                                away_normal[8] * train_data["home_xG"] + 
                                                away_normal[9] * train_data["home_xT"] +
                                                away_normal[10] * train_data["home_xD"] +
                                                away_normal[11] * train_data["home_xK"] +
                                                away_normal[12] * train_data["home_sub_xG"] + 
                                                away_normal[13] * train_data["home_sub_xT"] +
                                                away_normal[14] * train_data["home_sub_xD"] +
                                                away_normal[15] * train_data["home_sub_xK"])


    home_advantage = pm.Normal("home_advantage", mu=0, sigma=10)
    Intercept_home = pm.Normal("Intercept_home", mu=train_data["home_score"].mean(), sigma=0.5)
    Intercept_away = pm.Normal("Intercept_away", mu=train_data["away_score"].mean(), sigma=0.5)

    theta_home = pm.Deterministic("theta_home", np.exp(Intercept_home + home_advantage + home_team)) # log? / invlog?
    theta_away = pm.Deterministic("theta_away", np.exp(Intercept_away + away_team)) # log? / invlog?


    poisson_home = pm.Poisson("poisson_home", theta_home, observed=train_data["home_score"])
    poisson_away = pm.Poisson("poisson_away", theta_away, observed=train_data["away_score"])

In [ ]:
# N = train_data.shape[0]
# K = 2
# features = np.swapaxes(np.array([train_data["home_xG"], train_data["home_xT"], 
#                                  train_data["away_xG"], train_data["away_xT"], 
#                                  train_data["home_xD"], train_data["home_xK"], 
#                                  train_data["away_xD"], train_data["away_xK"]]), 0, 1)
# counts = np.swapaxes(np.array([train_data["home_score"], train_data["away_score"]]), 0, 1)
# P_features = features.shape[1]
# M = 10

# with pm.Model() as train_model:
#     # home_normal = pm.Normal("home_normal", mu=0, sigma=2, shape=2)
#     # away_normal = pm.Normal("away_normal", mu=0, sigma=2, shape=2)
    
#     # home_team = pm.Deterministic("home_team",   home_normal[0] * train_data["home_xG"] + 
#     #                                             home_normal[1] * train_data["home_xT"])

#     # away_team = pm.Deterministic("away_team",   away_normal[0] * train_data["away_xG"] + 
#     #                                             away_normal[1] * train_data["away_xT"])

#     coefs = pm.Normal('coefs', shape=(P_features, K))
#     cov_diag = pm.HalfNormal('cov_diag', shape=K)
#     cov_root = pm.Normal('cov_root', shape=(M, K))                                          
#     cov = pm.Deterministic('cov', cov_root.T @ cov_root + at.diag(cov_diag))

#     intercepts = pm.Normal('intercepts', shape=K)
#     # intercepts = pm.Normal('intercepts', mu=[train_data["home_score"].mean(), train_data["away_score"].mean()], sigma = 1, shape=K)
#     log_lam = pm.MvNormal("log_lam", mu = intercepts + features @ coefs, cov=cov, shape=(N, K))
#     lam = pm.math.exp(log_lam)

#     obs = pm.Poisson("obs", mu=lam, observed=counts)
#     # poisson_away = pm.Poisson("poisson_away", theta_away, observed=train_data["away_score"])

In [ ]:
# N = train_data.shape[0]
# K = 2
# features = np.swapaxes(np.array([train_data["home_xG"], train_data["home_xT"], 
#                                  train_data["away_xG"], train_data["away_xT"], 
#                                  train_data["home_xD"], train_data["home_xK"], 
#                                  train_data["away_xD"], train_data["away_xK"]]), 0, 1)
# counts = np.swapaxes(np.array([train_data["home_score"], train_data["away_score"]]), 0, 1)
# P_features = features.shape[1]
# M = 10

# with pm.Model() as train_model:  
#     sd_dist = pm.HalfNormal.dist(shape=K)
#     chol, corr, stds = pm.LKJCholeskyCov('chol_cov', n=K, eta=2, sd_dist=sd_dist, compute_corr=True)

#     coefs = pm.Normal('coefs', shape=(P_features, K))

#     intercepts = pm.Normal('intercepts', shape=K)
#     log_lam = pm.MvNormal('log_lam', mu=intercepts + features @ coefs, chol=chol, shape=(N,K))
#     lam = pm.math.exp(log_lam)
#     obs = pm.Poisson('obs', mu=lam, observed=counts)

In [ ]:
pm.model_to_graphviz(train_model)

In [ ]:
with train_model:
    # trace = pm.sample(1000, tune=1000, return_inferencedata=True, discard_tuned_samples=True)
    app = pm.fit(50000)
    trace = app.sample(1000) 

In [ ]:
# az.summary(trace.posterior, var_names=["intercepts", "coefs"], kind="stats")
summ = az.summary(trace.posterior, var_names=["Intercept_home", "Intercept_away", "home_advantage", "home_normal", "away_normal"], kind="stats")
summ['label'] = ["", "", "", "home_xG", "home_xT", "home_xD", "home_xK", 
                 "home_sub_xG", "home_sub_xT", "home_sub_xD", "home_sub_xK", "away_xG", "away_xT", 
                 "away_xD", "away_xK", "away_sub_xG", "away_sub_xT", "away_sub_xD", "away_sub_xK",
                 "away_xG", "away_xT", "away_xD", "away_xK", "away_sub_xG", "away_sub_xT", "away_sub_xD",
                 "away_sub_xK", "home_xG", "home_xT", "home_xD", "home_xK", "home_sub_xG", "home_sub_xT",
                 "home_sub_xD", "home_sub_xK"]
summ

# Test

In [ ]:
test_data = pd.read_csv("data_acquisition/data_game_values_test.csv", sep=";")
test_data_orig_size = test_data.shape[0]
size_diff = train_data.shape[0] - test_data.shape[0]
column_size = test_data.shape[1]
fill_data = [np.ones(column_size) for _ in range(size_diff)]
test_data_fill = pd.DataFrame(data=fill_data, columns=test_data.columns)
test_data = pd.concat([test_data, test_data_fill])

In [ ]:
with pm.Model() as test_model:
    home_normal = pm.Normal("home_normal", shape=16)
    away_normal = pm.Normal("away_normal", shape=16)
    
    home_team = pm.Deterministic("home_team", home_normal[0] * test_data["home_xG"] + 
                                                home_normal[1] * test_data["home_xT"] +
                                                home_normal[2] * test_data["home_xD"] +
                                                home_normal[3] * test_data["home_xK"] +
                                                home_normal[4] * test_data["home_sub_xG"] + 
                                                home_normal[5] * test_data["home_sub_xT"] +
                                                home_normal[6] * test_data["home_sub_xD"] +
                                                home_normal[7] * test_data["home_sub_xK"] +
                                                home_normal[8] * test_data["away_xG"] + 
                                                home_normal[9] * test_data["away_xT"] +
                                                home_normal[10] * test_data["away_xD"] +
                                                home_normal[11] * test_data["away_xK"] +
                                                home_normal[12] * test_data["away_sub_xG"] + 
                                                home_normal[13] * test_data["away_sub_xT"] +
                                                home_normal[14] * test_data["away_sub_xD"] +
                                                home_normal[15] * test_data["away_sub_xK"])

    away_team = pm.Deterministic("away_team", away_normal[0] * test_data["home_xG"] + 
                                                away_normal[1] * test_data["home_xT"] +
                                                away_normal[2] * test_data["home_xD"] +
                                                away_normal[3] * test_data["home_xK"] +
                                                away_normal[4] * test_data["home_sub_xG"] + 
                                                away_normal[5] * test_data["home_sub_xT"] +
                                                away_normal[6] * test_data["home_sub_xD"] +
                                                away_normal[7] * test_data["home_sub_xK"] +
                                                away_normal[8] * test_data["away_xG"] + 
                                                away_normal[9] * test_data["away_xT"] +
                                                away_normal[10] * test_data["away_xD"] +
                                                away_normal[11] * test_data["away_xK"] +
                                                away_normal[12] * test_data["away_sub_xG"] + 
                                                away_normal[13] * test_data["away_sub_xT"] +
                                                away_normal[14] * test_data["away_sub_xD"] +
                                                away_normal[15] * test_data["away_sub_xK"])

    home_advantage = pm.Normal("home_advantage", mu=0, sigma=10)
    Intercept_home = pm.Normal("Intercept_home", mu=train_data["home_score"].mean(), sigma=0.5)
    Intercept_away = pm.Normal("Intercept_away", mu=train_data["away_score"].mean(), sigma=0.5)

    theta_home = pm.Deterministic("theta_home", np.exp(Intercept_home + home_advantage + home_team)) # log? / invlog?
    theta_away = pm.Deterministic("theta_away", np.exp(Intercept_away + away_team)) # log? / invlog?


    poisson_home = pm.Poisson("poisson_home", theta_home, observed=test_data["home_score"])
    poisson_away = pm.Poisson("poisson_away", theta_away, observed=test_data["away_score"])

In [ ]:
# N = test_data.shape[0]
# K = 2

# features = np.swapaxes(np.array([test_data["home_xG"], test_data["home_xT"], 
#                                  test_data["away_xG"], test_data["away_xT"], 
#                                  test_data["home_xD"], test_data["home_xK"],
#                                  test_data["away_xD"], test_data["away_xK"]]), 0, 1)
# counts = np.swapaxes(np.array([test_data["home_score"], test_data["away_score"]]), 0, 1)
# P_features = features.shape[1]
# M = 10

# with pm.Model() as test_model:
#     # home_normal = pm.Normal("home_normal", mu=0, sigma=2, shape=2)
#     # away_normal = pm.Normal("away_normal", mu=0, sigma=2, shape=2)
    
#     # home_team = pm.Deterministic("home_team",   home_normal[0] * train_data["home_xG"] + 
#     #                                             home_normal[1] * train_data["home_xT"])

#     # away_team = pm.Deterministic("away_team",   away_normal[0] * train_data["away_xG"] + 
#     #                                             away_normal[1] * train_data["away_xT"])

#     coefs = pm.Normal('coefs', shape=(P_features, K))
#     cov_diag = pm.HalfNormal('cov_diag', shape=K)
#     cov_root = pm.Normal('cov_root', shape=(M, K))                                          
#     cov = pm.Deterministic('cov', cov_root.T @ cov_root + at.diag(cov_diag))

#     intercepts = pm.Normal('intercepts', shape=K)
#     log_lam = pm.MvNormal("log_lam", mu = intercepts + features @ coefs, cov=cov, shape=(N, K))
#     lam = pm.math.exp(log_lam)

#     obs = pm.Poisson("obs", mu=lam, observed=counts)
#     # poisson_away = pm.Poisson("poisson_away", theta_away, observed=train_data["away_score"])

In [ ]:
# N = test_data.shape[0]
# K = 2
# features = np.swapaxes(np.array([test_data["home_xG"], test_data["home_xT"], 
#                                  test_data["away_xG"], test_data["away_xT"], 
#                                  test_data["home_xD"], test_data["home_xK"], 
#                                  test_data["away_xD"], test_data["away_xK"]]), 0, 1)
# counts = np.swapaxes(np.array([test_data["home_score"], test_data["away_score"]]), 0, 1)
# P_features = features.shape[1]
# M = 10

# with pm.Model() as test_model:  
#     sd_dist = pm.HalfNormal.dist(shape=K)
#     chol, corr, stds = pm.LKJCholeskyCov('chol_cov', n=K, eta=2, sd_dist=sd_dist, compute_corr=True)

#     coefs = pm.Normal('coefs', shape=(P_features, K))

#     intercepts = pm.Normal('intercepts', shape=K)
#     log_lam = pm.MvNormal('log_lam', mu=intercepts + features @ coefs, chol=chol, shape=(N,K))
#     lam = pm.math.exp(log_lam)
#     obs = pm.Poisson('obs', mu=lam, observed=counts)

In [ ]:
sample_res = pm.sample_posterior_predictive(trace, test_model, predictions=True)
predictions = sample_res['predictions']

In [ ]:
actual_home = test_data[:test_data_orig_size].home_score
actual_away = test_data[:test_data_orig_size].away_score
act_res = []
for h, a in zip(actual_home, actual_away):
    act_res.append(f"{str(int(h))}:{str(int(a))}")


predictions_home = np.swapaxes(np.array(predictions.poisson_home[0].values), 0, 1)[:test_data_orig_size]# [:,:,0]
predictions_away = np.swapaxes(np.array(predictions.poisson_away[0].values), 0, 1)[:test_data_orig_size]# [:,:,1]

game_quotes = []
most_goals = {"home": [], "away": []}
for game_idx in range(len(predictions_home)):
    home_hist, bin_edges = np.histogram(predictions_home[game_idx], [0,1,2,3,4,5,6])
    away_hist, bin_edges = np.histogram(predictions_away[game_idx], [0,1,2,3,4,5,6])
    home, draw, away = 0, 0, 0
    for i in range(len(home_hist)):
        for j in range(len(away_hist)):
            if j < i:
                home += (home_hist[i]/1000) * (away_hist[j]/1000)
            elif j == i:
                draw += (home_hist[i]/1000) * (away_hist[j]/1000)
            elif j > i:
                away += (home_hist[i]/1000) * (away_hist[j]/1000)

    game_quotes.append(f"{round(home*100)}-{round(draw*100)}-{round(away*100)}")
    most_goals["home"].append(f"h: {np.argmax(home_hist)} - {round(np.max(home_hist)/10)}%")
    most_goals["away"].append(f"a: {np.argmax(away_hist)} - {round(np.max(away_hist)/10)}%")

df_res = pd.DataFrame({"actual": act_res, "predicted": game_quotes, "prob goals home" : most_goals['home'], "prob goals away" : most_goals['away']})
df_cross = pd.DataFrame({"actual": [0 if int(df_res.iloc[x]['actual'].split(':')[0]) > int(df_res.iloc[x]['actual'].split(':')[1]) 
                                    else 1 if int(df_res.iloc[x]['actual'].split(':')[0]) == int(df_res.iloc[x]['actual'].split(':')[1])
                                    else 2
                                    for x in range(df_res.shape[0])],
                         "pred":   [np.argmax(df_res.iloc[x]['predicted'].split('-'))
                                    for x in range(df_res.shape[0])]})

In [ ]:
# actual_home = test_data[:test_data_orig_size].home_score
# actual_away = test_data[:test_data_orig_size].away_score
# act_res = []
# for h, a in zip(actual_home, actual_away):
#     act_res.append(f"{str(int(h))}:{str(int(a))}")


# predictions_home = np.swapaxes(np.array(predictions.poisson_home[0].values), 0, 1)[:test_data_orig_size]
# predictions_away = np.swapaxes(np.array(predictions.poisson_away[0].values), 0, 1)[:test_data_orig_size]

# game_quotes = []
# most_goals = {"home": [], "away": []}
# for game_idx in range(len(predictions_home)):
#     home_hist, bin_edges = np.histogram(predictions_home[game_idx], [0,1,2,3,4,5,6])
#     away_hist, bin_edges = np.histogram(predictions_away[game_idx], [0,1,2,3,4,5,6])
#     home, draw, away = 0, 0, 0
#     for i in range(len(home_hist)):
#         for j in range(len(away_hist)):
#             if j < i:
#                 home += (home_hist[i]/1000) * (away_hist[j]/1000)
#             elif j == i:
#                 draw += (home_hist[i]/1000) * (away_hist[j]/1000)
#             elif j > i:
#                 away += (home_hist[i]/1000) * (away_hist[j]/1000)

#     game_quotes.append(f"{round(home*100)}-{round(draw*100)}-{round(away*100)}")
#     most_goals["home"].append(f"h: {np.argmax(home_hist)} - {round(np.max(home_hist)/10)}%")
#     most_goals["away"].append(f"a: {np.argmax(away_hist)} - {round(np.max(away_hist)/10)}%")

# df_res = pd.DataFrame({"actual": act_res, "predicted": game_quotes, "prob goals home" : most_goals['home'], "prob goals away" : most_goals['away']})
# df_cross = pd.DataFrame({"actual": [0 if int(df_res.iloc[x]['actual'].split(':')[0]) > int(df_res.iloc[x]['actual'].split(':')[1]) 
#                                     else 1 if int(df_res.iloc[x]['actual'].split(':')[0]) == int(df_res.iloc[x]['actual'].split(':')[1])
#                                     else 2
#                                     for x in range(df_res.shape[0])],
#                          "pred":   [np.argmax(df_res.iloc[x]['predicted'].split('-'))
#                                     for x in range(df_res.shape[0])]})

In [ ]:
print("Verteilung:")
print(f"Anzahl Predicted Home: {df_cross[df_cross['pred'] == 0].shape[0]} ({round(df_cross[df_cross['pred'] == 0].shape[0] / df_cross.shape[0] * 100, 2)}%)", end = '')
print(f" | Verteilung Tatsächlich Home: {round(df_cross[df_cross['actual'] == 0].shape[0] / df_cross.shape[0] * 100, 2)}%")
print(f"Anzahl Predicted Draw: {df_cross[df_cross['pred'] == 1].shape[0]} ({round(df_cross[df_cross['pred'] == 1].shape[0] / df_cross.shape[0] * 100, 2)}%)", end = '')
print(f" | Verteilung Tatsächlich Draw: {round(df_cross[df_cross['actual'] == 1].shape[0] / df_cross.shape[0] * 100, 2)}%")
print(f"Anzahl Predicted Away: {df_cross[df_cross['pred'] == 2].shape[0]} ({round(df_cross[df_cross['pred'] == 2].shape[0] / df_cross.shape[0] * 100, 2)}%)", end = '')
print(f" | Verteilung Tatsächlich Away: {round(df_cross[df_cross['actual'] == 2].shape[0] / df_cross.shape[0] * 100, 2)}%")
print("---------------------------------------------------------------------------------")
print("---------------------------------------------------------------------------------")
right, wrong, home_right, draw_right, away_right = 0, 0, 0, 0, 0
home_pred = {'act_away': 0, 'act_draw': 0}
draw_pred = {'act_away': 0, 'act_home': 0}
away_pred = {'act_home': 0, 'act_draw': 0}
for x in range(df_cross.shape[0]):
    if df_cross.iloc[x]["actual"] != df_cross.iloc[x]["pred"]:
        if df_cross.iloc[x]["pred"] == 0:
            if df_cross.iloc[x]["actual"] == 1:
                home_pred['act_draw'] += 1
            else:
                home_pred['act_away'] += 1
        elif df_cross.iloc[x]["pred"] == 1:
            if df_cross.iloc[x]["actual"] == 0:
                draw_pred['act_home'] += 1
            else:
                draw_pred['act_away'] += 1
        else:
            if df_cross.iloc[x]["actual"] == 0:
                away_pred['act_home'] += 1
            else:
                away_pred['act_draw'] += 1
        wrong += 1
    else:
        if df_cross.iloc[x]["actual"] == 0:
            home_right += 1
        elif df_cross.iloc[x]["actual"] == 1:
            draw_right += 1
        else:
            away_right += 1
        right += 1

home_wrong = home_pred['act_away'] + home_pred['act_draw']
draw_wrong = draw_pred['act_away'] + draw_pred['act_home']
away_wrong = away_pred['act_home'] + away_pred['act_draw']
print("Prediction:")
print(f"Anzahl Korrekt: {right} ({round(right / df_cross.shape[0] * 100,2)}%), Anzahl Falsch: {wrong} ({round(wrong / df_cross.shape[0] * 100, 2)}%)")
print("---------------------------------------------------------------------------------")
print(f"Anzahl Home Korrekt: {home_right} ({round(home_right / df_cross[df_cross['actual'] == 0].shape[0] * 100, 2)}%), Anzahl Home Falsch: {home_wrong}")
print(f"Home Pred. aber Draw --> {home_pred['act_draw']}")
print(f"Home Pred. aber Away --> {home_pred['act_away']}")
print(f"Anzahl Draw Korrekt: {draw_right} ({round(draw_right / df_cross[df_cross['actual'] == 1].shape[0] * 100, 2)}%), Anzahl Draw Falsch: {draw_wrong}")
print(f"Draw Pred. aber Home --> {draw_pred['act_home']}")
print(f"Draw Pred. aber Away --> {draw_pred['act_away']}")
print(f"Anzahl Away Korrekt: {away_right} ({round(away_right / df_cross[df_cross['actual'] == 2].shape[0] * 100, 2)}%), Anzahl Away Falsch: {away_wrong}")
print(f"Away Pred. aber Home --> {away_pred['act_home']}")
print(f"Away Pred. aber Draw --> {away_pred['act_draw']}")



In [ ]:
home_histograms, away_histograms = [], []
for game_idx in range(len(predictions_home)):
    home_hist, bin_edges = np.histogram(predictions_home[game_idx], [0,1,2,3,4,5,6])
    away_hist, bin_edges = np.histogram(predictions_away[game_idx], [0,1,2,3,4,5,6])
    home_histograms.append(home_hist)
    away_histograms.append(away_hist)

home_cum_hist = np.sum(home_histograms, axis=0)
away_cum_hist = np.sum(away_histograms, axis=0)

In [ ]:
his_h, _ = np.histogram(train_data.home_score, [0,1,2,3,4,5,6])
his_a, _ = np.histogram(train_data.away_score, [0,1,2,3,4,5,6])
fig, (ax1, ax2) = plt.subplots(2, 2)
ax1[0].bar(np.arange(len(his_h)),his_h / np.sum(his_h))
ax1[0].set_title("Home Actual")
ax1[0].grid(axis='y')

ax2[0].bar(np.arange(len(home_cum_hist)),home_cum_hist / np.sum(home_cum_hist), color='r')
ax2[0].set_title("Home Pred")
ax2[0].grid(axis='y')

ax1[1].bar(np.arange(len(his_a)),his_a / np.sum(his_a))
ax1[1].set_title("Away Actual")
ax1[1].grid(axis='y')

ax2[1].bar(np.arange(len(away_cum_hist)),away_cum_hist / np.sum(away_cum_hist), color='r')
ax2[1].set_title("Away Pred")
ax2[1].grid(axis='y')